<a href="https://colab.research.google.com/github/achri19/BAM/blob/main/notebooks/Downloading_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="8">Preparations: Downloading Ancillary Data</font>

<font size="3">In this notebook, we will:

- Download GEBCO, Global Mangrove Watch, and Geoid datasets that are needed for the tutorials. 
- You only need to run this notebook once to download the files, but it will take a long time and requires 15+ GB of space on your hard drive.


<font size=5 color='green'> If you are running in Google Colab, set the variable yes_colab = True.</font> <br>
<font size=5 color='blue'> If you are running on your own computer, set the variable yes_colab = False </font>


In [1]:
yes_colab = False


<font size=6> Step #1: Set working directory <br> </font>



<font size=5 color='green'> If you are running in Google Colab, when you run the next cell, a pop-up window will appear asking you to grant access to your Google Drive. You must approve or the notebook will not work. <font> <br>



In [2]:
import os
import sys
from pathlib import Path

your_path = Path(os.getcwd() + '/')
if yes_colab:
  where_to_mount = '/content/drive/'
  from google.colab import drive
  drive.mount(where_to_mount, force_remount=True)
  mounted_drive = Path(where_to_mount) / 'MyDrive' 
  sys.path.append(str(mounted_drive / 'installations'))
  path_ancillary = mounted_drive / 'ancillary'
  Path(mounted_drive / 'installations').mkdir(parents=True, exist_ok=True)

else:
  mounted_drive = Path(os.path.abspath(os.path.join(your_path, os.pardir)))
  path_ancillary = mounted_drive / 'ancillary'

print('Your working directory is %s' %(mounted_drive))



Your working directory is C:\Users\alchrist\Documents\BAM


<font size=6> Step #2: Install and import packages. <font> <br>

<font size=5 color='green'> If you are running in Google Colab, this cell should install all Python packages you need for each tutorial. </font> <br>
<font size=5 color='blue'> If you are running on your own computer, the packages were already installed when you installed the conda environment </font>


In [3]:
import os
os.chdir(mounted_drive)
if yes_colab:
  if os.path.isdir(mounted_drive / 'BAM'):
    print('## Updating the local git repository \n')
    os.chdir(mounted_drive / 'BAM')
    #!git fsck --full
    # !git add -A 
    !git stash   
    !git pull
    # !rm -rf {mounted_drive/'BAM'}

  else:
    print('## Pulling the git repository with files for the tutorial\n')
    !git clone https://github.com/achri19/BAM.git


In [4]:
if yes_colab:
  print('\n## Installing the Python packages needed for these tutorials\n')
  try:
    import geopandas as gpd
  except:
    !/bin/bash $mounted_drive/BAM/notebooks/install_packages_colab_debug.sh


In [5]:
path_ancillary = mounted_drive / 'ancillary'
path_code = mounted_drive / 'scripts/'
path_templates = mounted_drive /  'templates/'
path_configs = mounted_drive / 'configs/'
sys.path.insert(1,str(path_code))
print(path_code)
print(path_ancillary)

path_examples = mounted_drive / 'examples'
Path(path_examples).mkdir(parents=True, exist_ok=True)

C:\Users\alchrist\Documents\BAM\scripts
C:\Users\alchrist\Documents\BAM\ancillary


In [6]:
import sys
import os
import pandas as pd
import shutil
from datetime import datetime
from string import Template
import fnmatch
import geopandas as gpd
import rasterio
from osgeo import gdal 
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import rtree
from scipy import *


C:\Users\alchrist\AppData\Local\Temp\ipykernel_17804\1982723361.py:8: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [ ]:
try:
  import pyTMD
except:
  !pip install pytmd
try:
    import skimage.measure as measure
except:
    !pip install scikit-image
    
from main_tools import (build_directory,
                       get_extent_parameters,
                       setup_AOI_files, 
                       make_polygons,
                       make_channel_networks,
                       make_model_foundation, 
                       set_boundary_conditions, 
                       make_watermask,
                       more_opening)


<font size=6> Step #3: Download to your ancillary folder <br> </font>



In [18]:
import urllib.request
import zipfile
import shutil

if os.path.isfile(path_ancillary / 'geoids' / 'egm08_25.gtx')==False:
  url = 'https://download.osgeo.org/proj/vdatum/egm08_25/egm08_25.gtx'
  filename = path_ancillary / 'geoids' / 'egm08_25.gtx'
  urllib.request.urlretrieve(url, filename)


if os.path.isfile(path_ancillary / 'gebco' / 'gebco_all.vrt')==False:
  url = 'https://www.bodc.ac.uk/data/open_download/gebco/gebco_2022_sub_ice_topo/geotiff/'
  filename = path_ancillary / 'gebco' / 'gebco.zip'
#   urllib.request.urlretrieve(url, filename)

  with zipfile.ZipFile(path_ancillary / 'gebco' / 'gebco.zip', 'r') as zip_ref:
      zip_ref.extractall(path_ancillary / 'gebco' )

  gebco_files = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(path_ancillary / 'gebco')
                for f in fnmatch.filter(files,'*gebco_2022*.tif')]
  print(gebco_files)

  with open(path_ancillary / 'gebco' / 'gebco_files.txt', 'w') as f:
    for item in gebco_files:
      f.write("%s\n" % item)
  ! gdalbuildvrt -input_file_list {path_ancillary}/gebco/gebco_files.txt {path_ancillary}/gebco/gebco_all.vrt
try: os.remove(path_ancillary/'gebco'/'gebco.zip')
except:''

if os.path.isfile(path_ancillary / 'gmw' / 'gmw_v3_2016_vec.shp')==False:
  url = 'https://zenodo.org/record/6894273/files/gmw_v3_2016_vec.zip?download=1'
  filename = path_ancillary / 'gmw' / 'gmw_v3_2016_vec.zip'
  urllib.request.urlretrieve(url, filename)

  with zipfile.ZipFile(path_ancillary / 'gmw' / 'gmw_v3_2016_vec.zip', 'r') as zip_ref:
      zip_ref.extractall(path_ancillary / 'gmw')

try:os.remove(path_ancillary/'gmw'/'gmw_v3_2016_vec.zip')
except:''

if os.path.isfile(path_ancillary / 'hydropolys' / 'hydropolys.shp')==False:
  url = 'http://faculty.baruch.cuny.edu/geoportal/data/esri/world/hydropolys.zip'
  filename = path_ancillary / 'hydropolys' / 'hydropolys.zip'
  urllib.request.urlretrieve(url, filename)

  with zipfile.ZipFile(path_ancillary / 'hydropolys' / 'hydropolys.zip', 'r') as zip_ref:
      zip_ref.extractall(path_ancillary / 'hydropolys')

try:os.remove(path_ancillary/'hydropolys'/'hydropolys.zip')
except:''            

